In [1]:
## libraries 

import subprocess
import anndata
import scanpy as sc
import pandas as pd
import os
import scEntropy.scEntropy as scEntropy
import csv
import numpy as np
from matplotlib import pyplot as plt 

In [2]:
## functions 
def adataX2pandas(adata):
    gene_expression = pd.DataFrame(adata.X)
    gene_expression.columns = adata.var.index 
    gene_expression.index = adata.obs.index
    gene_expression = gene_expression.T
    return(gene_expression)

def Generating_data_for_I_go(adata):
    gene_expression_df = pd.DataFrame()
    gene_expression_df = pd.concat([gene_expression_df, pd.DataFrame(adata.X)], axis=1)
    gene_expression_df.columns = [adata.var.index]
    gene_expression_df.to_csv('data/gene_expression.csv')
    adata.obs.to_csv("data/adataobs.csv")
    adata.var.to_csv("data/adatavar.csv")

# calculating E, S, C
def calculating_ESC(adata, I_ge_label="I_ge", I_go_label="I_go"):
    
    E = adata.obs[I_go_label]/adata.obs[I_ge_label]
    S = adata.obs[I_ge_label] - adata.obs[I_go_label]
    C = E * S
    
    adata.obs['E'] = E 
    adata.obs['S'] = S
    adata.obs['C'] = C 
    
    
    return adata

def Returning_I_go(adata):

    # 1. Generating relavent files
    Generating_data_for_I_go(adata)

    # 2. Running R script
    subprocess.call("R CMD BATCH REntropyIgo.r", shell=True)

    # 3. loading data
    I_go = pd.read_csv("data/Rigo.csv")

    return(I_go)

In [4]:
## start of script 

## finding Ige 

## loading adata object
adata = sc.read_h5ad("data/TrajectInfer.h5ad")
print("loading adata object")

loading adata object


# Getting I_ge

In [ ]:
# Getting I_ge (scEntropy)
# I_ge = scEntropy.scEntropy(adataX2pandas(adata), option='RCSA')
# print("Getting I_ge")

In [26]:
# ad hoc attempt

adata_raw = sc.datasets.paul15()
print(adata_raw)

AnnData object with n_obs × n_vars = 2730 × 3451
    obs: 'paul15_clusters'
    uns: 'iroot'


/home/balubao/anaconda3/envs/EntropyV2/lib/python3.10/site-packages/scanpy/datasets/_datasets.py:202: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  adata.uns['iroot'] = 840


In [54]:
gene_expression_df = pd.DataFrame(adata_raw.X)

# gene_expression_df.iloc()/np.sum(gene_expression_df, axis=1)[1]
prob = gene_expression_df.div(gene_expression_df.sum(), axis=1)
logprob = np.log(prob)
logprob[np.isinf(logprob)]=0

scent = -np.sum(prob*logprob, axis=1)/prob.shape[
print(scent)

0       0.000534
1       0.003344
2       0.005469
3       0.005849
4       0.005721
          ...   
2725    0.004579
2726    0.004595
2727    0.000729
2728    0.004505
2729    0.006359
Length: 2730, dtype: float32


/home/balubao/anaconda3/envs/EntropyV2/lib/python3.10/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [55]:
adata.obs["I_ge"] = scent

# Getting I_go 

In [ ]:
# Getting I_go 
# I_go = Returning_I_go(adata)
# print("Getting I_go")

In [ ]:
## generate entropies at different coarse grains
##DEMO
import random

categories = ["A", "B", "C", "D", "E"]

# Generate a random array of categorical labels
gene_group_labels = [random.choice(categories) for _ in range(adata_raw.shape[1])]  # Replace 10 with the desired length
len(gene_group_labels)

## by GO scale

In [87]:
gene_expression_pp_df = np.log1p(gene_expression_df.div(gene_expression_df.sum(), axis=1)*1e4)

In [94]:
## Might be easier to compute in R where WGCNA is established. Perhaps even create independent conda env for it.
thresh = 0.3 #there are methods to choose parameters based on scale-free topology assumption.
beta = 5

GCN = np.abs(np.corrcoef(gene_expression_pp_df.transpose())) #more robust methods include spearman and biweight midcorrelation - see WGCNA
GCN[GCN<thresh] = 0
GCN_pp = GCN**beta 
GCN_pp




array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.00487317,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.0045517 ,
        0.        ],
       [0.        , 0.        , 0.00487317, ..., 0.0045517 , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [67]:
## by GCN clustering scale

# use something similar to WGCNA, then slice heirarical clustering tree at different heights.
# 100-70-50-20 modules.

def CompressMatrixByLabels(matrix, labels):

    # Get unique categories from the label array
    unique_categories = np.unique(labels)
    
    # Create an empty dictionary to hold the indices of each category
    category_indices = {category: [] for category in unique_categories}
    
    # Populate the dictionary with indices
    for idx, category in enumerate(labels):
        category_indices[category].append(idx)
    
    # Calculate the mean for each category
    compressed_matrix = np.zeros((matrix.shape[0], len(unique_categories)))
    for category, indices in category_indices.items():
        category_mean = np.mean(matrix[:, indices], axis=1)
        compressed_matrix[:, unique_categories == category] = category_mean[:, np.newaxis]
        
    return compressed_matrix

coarsed_gene_df = CompressMatrixByLabels(gene_expression_df.to_numpy(), gene_group_labels)
coarsed_gene_df.shape

(2730, 5)

In [72]:
# import louvain
# import leidenalg
import seaborn

# Compute Downstream metrics

In [ ]:
# inscribing I_go & I_ge into adata.obs 
adata.obs["I_ge"] = I_ge
adata.obs["I_go"] = I_go["scEntropy.bootstrap"].to_numpy()
# adata.obs = pd.concat([adata.obs, I_ge], axis=1)

# adata.write("C:/Users/Dr. Hamad/Downloads/SRSI/SRSI projects/data/TrajectInfer.h5ad")
 

adata = calculating_ESC(adata, I_ge_label="I_ge", I_go_label="I_go")
print("Calculating E,S,C")

In [ ]:
# saving adata.with new variables 
adata.write_h5ad("data/Paul15ESC.h5ad")

# 